<img src="./pictures/logo_sizinglab.png" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Frame selection
*Written by Marc Budinger (INSA Toulouse) and Scott Delbecq (ISAE-SUPAERO), Toulouse, France.*

## Design graph 

The following diagram represents the design graph of the frame selection. 


![DesignGraph](pictures/FrameDesignGraph.png)

> **Questions:**
* Give the main sizing problems you are able to detect.
* Propose one or multiple solutions (which can request equation manipulation, addition of design variables, addition of constraints) 
* Orientate the arrows
* Give equations order, inputs/outputs at each step of this part of sizing procedure



### Sizing code and optimization

> Exercice: propose a sizing code for the selection of the frame.


In [1]:
from utils.model_standard import CoreModel
from utils.model_serializer import ModelSerializer

In [2]:
import math
from math import pi
import scipy

class FrameModel(CoreModel):
    """
    Frame model class.
    ----------
    """
    
    def initialization(self):
        
        # Input variables
        # ---------------
        # Narm [-] Arms number
        # Dpro [m] Propeller diameter 
        # Np_arm [-] Number of propellers per arm (1 or 2)
        # Tpro_takeoff [N] Thrust for one propeller during take off
        # k_frame [-] ratio thickness e of tube / diameter of tube
        
        self.add_input('N_arm', value=4.0, unit='-', comment='Number of arms')
        self.add_input('D_pro', value=0.3, unit='m', comment='Propeller diameter')
        self.add_input('N_pro_arm', value=1.0, unit='-', comment='Number of propellers per arm (1 or 2)')
        self.add_input('F_pro_to', value=1.0, unit='N', comment='Thrust for one propeller during take off')
        self.add_input('k_D', value=0.1, unit='N', comment='Ratio thickness / outer diameter of tube')
        
        # Input parameters
        
        # Static stress
        # sigma_max=200e6/4 # [Pa] Alu max stress (2 reduction for dynamic, 2 reduction for stress concentration)
        sigma_max = 280e6/4. # [Pa] Composite max stress (2 reduction for dynamic, 2 reduction for stress concentration)
        # rho_s = 2700. # [kg/m3] Volumic mass of aluminum
        rho_s = 1700. # [kg/m3] Volumic mass of aluminum
        
        self.add_input('sigma_max', value=sigma_max, unit='Pa', comment='Max admisible stress')
        self.add_input('rho_s', value=rho_s, unit='kg/m3', comment='Volumic mass of structure')
        
        # Declare outputs
        self.add_output('alpha_sep', unit='rad', comment='Interior angle separation between propellers')
        self.add_output('L_arm', unit='m', comment='Length of the arm')
        self.add_output('D_out_arm', unit='m', comment='Outer diameter of the arm (tube)')
        self.add_output('D_in_arm', unit='m', comment='Inner diameter of the arm (tube)')
        self.add_output('e_arm', unit='m', comment='Thickness of the arm (tube)')
        self.add_output('M_arm', unit='kg', comment='Arm mass')
        self.add_output('M_frame', unit='kg', comment='Frame mass')

    def computation_script(self):
        p = self.parameters
        
        # Inputs
        N_arm = p['N_arm']
        D_pro = p['D_pro']
        F_pro_to = p['F_pro_to']
        N_pro_arm = p['N_pro_arm']
        sigma_max = p['sigma_max']
        k_D = p['k_D']
        rho_s = p['rho_s']
        
        # Length calculation   
        alpha_sep = 2*pi / N_arm # [rad] interior angle separation between propellers
        L_arm = D_pro / (2.*scipy.sin(alpha_sep/2.)) # [m] length of the arm

        # Tube diameter & thickness
        D_out_arm = (F_pro_to*N_pro_arm/sigma_max*L_arm*32./(pi*(1.-(1.-2.*k_D)**4.)))**(1/3)  # [m] outer diameter of the arm (hollow cylinder)
        e_arm = k_D * D_out_arm # [m] thickness of the arm (hollow cylinder) 
        D_in_arm = D_out_arm - e_arm # [m] inner diameter of the arm (hollow cylinder) 
        
        # Mass
        M_arm = pi/4. * (D_out_arm**2.-(D_out_arm-2.*e_arm)**2)*L_arm*rho_s # [kg] mass of the arm (x1) (beams only)  composite
        M_frame = N_arm * M_arm / 0.4 # [kg] mass of the frame (40% of total mass is the arms)
        
        # Outputs
        p['alpha_sep'] = alpha_sep
        p['L_arm'] = L_arm
        p['D_out_arm'] = D_out_arm
        p['e_arm'] = e_arm
        p['D_in_arm'] = D_in_arm
        p['M_arm'] = M_arm
        p['M_frame'] = M_frame

In [3]:
frame_model = FrameModel('frame')
frame_model.initialization()

frame_model.print_variables()

ms = ModelSerializer()
path = './models/'
file_name = 'frame_model'
ms.save_model(frame_model, path + file_name)

interactive(children=(Dropdown(description='Component', options=('frame',), value='frame'), Output()), _dom_cl…

In [4]:
inputs = {'F_pro_to': 25.0}

frame_model.compute(inputs)

frame_model.print_variables()

interactive(children=(Dropdown(description='Component', options=('frame',), value='frame'), Output()), _dom_cl…